In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

file_path = '/Volumes/main/default/medical_data_volume/medical_data_complete_unzipped/medical_data_local/drug-particle-identification-info/all_drug_data.json'

spark = SparkSession.builder.getOrCreate()

# Use multiline option and specify encoding
df = spark.read.option("multiline", "true").option("encoding", "UTF-8").json(file_path)

print("Schema:")
df.printSchema()

In [0]:
# Get basic dataset information
print("Dataset Overview:")
print(f"Total number of records: {df.count()}")
print(f"Number of columns: {len(df.columns)}")

# Show first few records
print("\nFirst 5 records:")
df.show(5, truncate=False)

# Get summary statistics for key columns
print("\nSummary statistics:")
df.describe().show()

# Check for null values in each column
print("\nNull value counts per column:")
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])
null_counts.show()

# Explore categorical columns - let's look at some key ones
print("\nUnique values in key categorical columns:")

# Drug classifications
print("ETC_OTC_NAME (Prescription/OTC status):")
df.groupBy("ETC_OTC_NAME").count().orderBy(desc("count")).show()

print("DRUG_SHAPE (Drug shapes):")
df.groupBy("DRUG_SHAPE").count().orderBy(desc("count")).show(20)

print("COLOR_CLASS1 (Primary colors):")
df.groupBy("COLOR_CLASS1").count().orderBy(desc("count")).show(20)

print("FORM_CODE_NAME (Form types):")
df.groupBy("FORM_CODE_NAME").count().orderBy(desc("count")).show()

# Check for duplicates
print(f"\nTotal records: {df.count()}")
print(f"Distinct records: {df.distinct().count()}")
print(f"Duplicate records: {df.count() - df.distinct().count()}")

# Look at some specific interesting columns
print("\nSample of drug names and companies:")
df.select("ITEM_NAME", "ENTP_NAME", "ITEM_ENG_NAME").show(10, truncate=False)

# Check date ranges
print("\nDate information:")
df.select("ITEM_PERMIT_DATE", "CHANGE_DATE", "IMG_REGIST_TS").show(5, truncate=False)

In [0]:
# Define your bronze table name
bronze_table_name = "main.default.drug_particle_identification_bronze"

# Write the DataFrame to a Delta table
print(f"Creating bronze table: {bronze_table_name}")

df.write \
  .format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .saveAsTable(bronze_table_name)

# Get total records count
total_records = df.count()
print(f"✅ Successfully created bronze table with {total_records} records")

# Verify the table was created
print(f"\nVerifying bronze table:")
bronze_df = spark.table(bronze_table_name)
print(f"Records in bronze table: {bronze_df.count()}")

# Show table info
print(f"\nBronze table schema:")
bronze_df.printSchema()

# Show sample data from bronze table
print(f"\nSample data from bronze table:")
bronze_df.select("ENTP_NAME", "ITEM_NAME", "ITEM_SEQ", "DRUG_SHAPE", "COLOR_CLASS1").show(5, truncate=False)

Bronze Table Explore

In [0]:
# Load the bronze table
bronze_df = spark.table("main.default.drug_particle_identification_bronze")

print("=== BASIC TABLE INFO ===")
print(f"Total records: {bronze_df.count()}")
print(f"Total columns: {len(bronze_df.columns)}")

print("\n=== COLUMN NAMES ===")
for i, col in enumerate(bronze_df.columns, 1):
    print(f"{i:2d}. {col}")

print("\n=== SAMPLE RECORDS ===")
bronze_df.show(5, truncate=False)

print("\n=== DATA QUALITY CHECK - NULL VALUES ===")
from pyspark.sql.functions import col, sum as spark_sum, when, isnan, isnull

# Check for null/empty values in key columns
key_columns = ["ITEM_NAME", "ENTP_NAME", "ITEM_SEQ", "DRUG_SHAPE", "COLOR_CLASS1"]
null_counts = []

for column in key_columns:
    null_count = bronze_df.select(
        spark_sum(
            when(col(column).isNull() | (col(column) == "") | (col(column) == "null"), 1).otherwise(0)
        ).alias("null_count")
    ).collect()[0]["null_count"]
    null_counts.append((column, null_count))
    
print("Column Name          | Null/Empty Count")
print("-" * 40)
for col_name, null_count in null_counts:
    print(f"{col_name:<20} | {null_count}")

print("\n=== UNIQUE VALUES IN KEY CATEGORICAL COLUMNS ===")

# Drug shapes
print("Drug Shapes:")
bronze_df.select("DRUG_SHAPE").filter(col("DRUG_SHAPE").isNotNull() & (col("DRUG_SHAPE") != "")).distinct().show(20, truncate=False)

# Color classes
print("\nPrimary Colors (COLOR_CLASS1):")
bronze_df.select("COLOR_CLASS1").filter(col("COLOR_CLASS1").isNotNull() & (col("COLOR_CLASS1") != "")).distinct().show(20, truncate=False)

print("\nSecondary Colors (COLOR_CLASS2):")
bronze_df.select("COLOR_CLASS2").filter(col("COLOR_CLASS2").isNotNull() & (col("COLOR_CLASS2") != "")).distinct().show(20, truncate=False)

print("\n=== TOP ENTERPRISES BY DRUG COUNT ===")
bronze_df.groupBy("ENTP_NAME") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10, truncate=False)

print("\n=== FORM CODE DISTRIBUTION ===")
bronze_df.groupBy("FORM_CODE_NAME") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10, truncate=False)

print("\n=== ETC/OTC DISTRIBUTION ===")
bronze_df.groupBy("ETC_OTC_NAME") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10, truncate=False)

print("\n=== SAMPLE DETAILED RECORDS ===")
print("Let's look at some complete records with interesting characteristics:")

# Show records with marking codes
bronze_df.select(
    "ITEM_NAME", 
    "ENTP_NAME", 
    "DRUG_SHAPE", 
    "COLOR_CLASS1", 
    "MARK_CODE_FRONT", 
    "MARK_CODE_BACK",
    "PRINT_FRONT",
    "PRINT_BACK"
).filter(
    (col("MARK_CODE_FRONT").isNotNull() & (col("MARK_CODE_FRONT") != "")) |
    (col("MARK_CODE_BACK").isNotNull() & (col("MARK_CODE_BACK") != ""))
).show(5, truncate=False)

print("\n=== IMAGE DATA AVAILABILITY ===")
image_stats = bronze_df.select(
    spark_sum(when(col("ITEM_IMAGE").isNotNull() & (col("ITEM_IMAGE") != ""), 1).otherwise(0)).alias("has_item_image"),
    spark_sum(when(col("MARK_CODE_FRONT_IMG").isNotNull() & (col("MARK_CODE_FRONT_IMG") != ""), 1).otherwise(0)).alias("has_front_mark_image"),
    spark_sum(when(col("MARK_CODE_BACK_IMG").isNotNull() & (col("MARK_CODE_BACK_IMG") != ""), 1).otherwise(0)).alias("has_back_mark_image")
).collect()[0]

total_records = bronze_df.count()
print(f"Records with item images: {image_stats['has_item_image']} ({image_stats['has_item_image']/total_records*100:.1f}%)")
print(f"Records with front mark images: {image_stats['has_front_mark_image']} ({image_stats['has_front_mark_image']/total_records*100:.1f}%)")
print(f"Records with back mark images: {image_stats['has_back_mark_image']} ({image_stats['has_back_mark_image']/total_records*100:.1f}%)")